In [1]:
# -*- coding: utf-8 -*-
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import re
from string import digits
import nltk; nltk.download('stopwords')
import os
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import pandas as pd
hotel_review = pd.read_csv('Hotel_Reviews.csv')
hotel_review.head()

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,26,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.360576,4.915968
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/24/2017,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,8,3,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.360576,4.915968


In [3]:
# 나라 중복값 확인
w_count  = {}
for lst in hotel_review['Reviewer_Nationality']:
    try: w_count[lst] += 1
    except: w_count[lst]=1
print(w_count)

{' Russia ': 3900, ' Ireland ': 14827, ' Australia ': 21686, ' United Kingdom ': 245246, ' New Zealand ': 3237, ' Poland ': 2891, ' Belgium ': 6031, ' Norway ': 2383, ' France ': 7296, ' Italy ': 6114, ' Canada ': 7894, ' Netherlands ': 8772, ' Germany ': 7941, ' United States of America ': 35437, ' Hungary ': 2185, ' Israel ': 6610, ' Switzerland ': 8678, ' India ': 3437, ' United Arab Emirates ': 10235, ' Malaysia ': 1883, ' Denmark ': 1724, ' South Africa ': 3821, ' Rwanda ': 13, ' Finland ': 1829, ' Portugal ': 1839, ' Luxembourg ': 1359, ' Spain ': 4737, ' Malta ': 1685, ' China ': 3400, ' Turkey ': 5444, ' Ukraine ': 1102, ' Gibraltar ': 380, ' Argentina ': 517, ' Abkhazia Georgia ': 153, ' Isle of Man ': 405, ' Sweden ': 3368, ' Croatia ': 1340, ' Saudi Arabia ': 8951, ' Lithuania ': 727, ' Slovenia ': 796, ' Panama ': 122, ' Austria ': 2412, ' Brazil ': 1865, ' Taiwan ': 923, ' Iceland ': 933, ' Iran ': 1086, ' Liechtenstein ': 20, ' Chile ': 334, ' Hong Kong ': 3013, ' United 

In [4]:
# 대륙별 행 추출
hotel_review_America = hotel_review[['Reviewer_Nationality',"Negative_Review","Positive_Review"]]
Usa = hotel_review_America[hotel_review_America['Reviewer_Nationality']  == " United Stated of America "]
Ca = hotel_review_America[hotel_review_America['Reviewer_Nationality']  == " Canada "]
Ar = hotel_review_America[hotel_review_America['Reviewer_Nationality']  == " Argentina "]
Br = hotel_review_America[hotel_review_America['Reviewer_Nationality']  == " Brazil "]
Me = hotel_review_America[hotel_review_America['Reviewer_Nationality']  == " Mexico "]
AMERICA = pd.concat([Usa,Ca,Ar,Br,Me])
AMERICA

,Reviewer_Nationality,Negative_Review,Positive_Review
14,Canada,The staff in the restaurant could of been mor...,It was very good very historic building that ...
39,Canada,Staff a few were friendly and willing enough ...,Location on the park with easy access to tram...
106,Canada,The front desk staff particularly the young m...,The rooms were comfortable and clean and the ...
117,Canada,The outside of the hotel is under constructio...,The inside of the hotel is very classy He sta...
119,Canada,The hotel gardens and patios were under major...,The location of this hotel is great It s very...
215,Canada,No Negative,It was a well kept hotel in a lovely area The...
219,Canada,We don t like the open concept shower It gets...,We liked the location and the extra pleasant ...
246,Canada,Beds sucked Air conditioner too loud for use,Nice property and building
264,Canada,The hotel was under renovation while we there...,The bed was very comfortable and the room was...
418,Canada,Squeaky floors Difficult to go to the bathroo...,Clean comfy beds Beautiful gardens Excellent ...


In [0]:
Negative = AMERICA.iloc[:,1]
Positive = AMERICA.iloc[:,2]

In [6]:
Negative = [re.sub("\'", "", str(sent)) for sent in Negative]
Negative = [re.sub("No Negative'", "", str(sent)) for sent in Negative]
pprint(Negative[:1])

[' The staff in the restaurant could of been more pleasant we only visited '
 'once but that wouldn t stop us from booking again it might of just been an '
 'off night for him ']


In [7]:
Positive = [re.sub("\'", "", str(sent1)) for sent1 in Positive]
Positive = [re.sub("No Positive'", "", str(sent1)) for sent1 in Positive]
pprint(Positive[:1])

[' It was very good very historic building that s why I chose it ']


In [0]:
import gensim 
from gensim.models import Word2Vec

In [9]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words1 = list(sent_to_words(Negative))
data_words2 = list(sent_to_words(Positive))

print(data_words1[:1])
print(data_words2[:1])

[['the', 'staff', 'in', 'the', 'restaurant', 'could', 'of', 'been', 'more', 'pleasant', 'we', 'only', 'visited', 'once', 'but', 'that', 'wouldn', 'stop', 'us', 'from', 'booking', 'again', 'it', 'might', 'of', 'just', 'been', 'an', 'off', 'night', 'for', 'him']]
[['it', 'was', 'very', 'good', 'very', 'historic', 'building', 'that', 'why', 'chose', 'it']]


In [10]:
%%time
model1 = Word2Vec(data_words1)

CPU times: user 1.96 s, sys: 16 ms, total: 1.98 s
Wall time: 1.15 s


In [11]:
%%time
model2 = Word2Vec(data_words2)

CPU times: user 1.78 s, sys: 17 ms, total: 1.79 s
Wall time: 1 s


In [13]:
model1.wv.most_similar("service")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('food', 0.9849331974983215),
 ('bad', 0.9822552800178528),
 ('restaurant', 0.9796799421310425),
 ('impersonal', 0.9785584807395935),
 ('slow', 0.978010892868042),
 ('wifi', 0.9748818874359131),
 ('pool', 0.9738884568214417),
 ('cooked', 0.9729845523834229),
 ('high', 0.9724013805389404),
 ('fitted', 0.9714035391807556)]

In [14]:
model1.wv.most_similar("room")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('elevator', 0.9537550210952759),
 ('bed', 0.944840669631958),
 ('bathroom', 0.937239408493042),
 ('side', 0.9328557252883911),
 ('size', 0.9327760338783264),
 ('shower', 0.9318338632583618),
 ('beds', 0.9309026598930359),
 ('dark', 0.9299718737602234),
 ('noisy', 0.9256476163864136),
 ('uncomfortable', 0.9234119653701782)]

In [15]:
model1.wv.most_similar("location")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('quite', 0.9838259220123291),
 ('old', 0.9787646532058716),
 ('dated', 0.9736653566360474),
 ('soft', 0.9725807309150696),
 ('high', 0.9722407460212708),
 ('far', 0.9702354669570923),
 ('fancy', 0.9687228798866272),
 ('bit', 0.9686322212219238),
 ('area', 0.9685993194580078),
 ('tight', 0.9682952761650085)]

In [17]:
model1.wv.most_similar("transport")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('helped', 0.9989373683929443),
 ('issue', 0.9989067912101746),
 ('tables', 0.9988416433334351),
 ('quickly', 0.9988182783126831),
 ('badly', 0.9987307786941528),
 ('directly', 0.998630166053772),
 ('inconvenience', 0.998599648475647),
 ('wrong', 0.9985870122909546),
 ('white', 0.9985445737838745),
 ('nor', 0.998522162437439)]

In [18]:
model2.wv.most_similar("service")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('restaurant', 0.9681097269058228),
 ('incredibly', 0.9608511924743652),
 ('concierge', 0.96083003282547),
 ('delicious', 0.9572252631187439),
 ('buffet', 0.9553201198577881),
 ('welcoming', 0.9529483318328857),
 ('professional', 0.952130913734436),
 ('breakfast', 0.9499074220657349),
 ('reception', 0.9483577013015747),
 ('facilities', 0.9463878870010376)]

In [21]:
model2.wv.most_similar("room")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('bathroom', 0.9665687084197998),
 ('bed', 0.9595426321029663),
 ('shower', 0.9514538049697876),
 ('sofa', 0.9425439834594727),
 ('large', 0.9311589002609253),
 ('size', 0.9281160831451416),
 ('pillows', 0.9269039630889893),
 ('spacious', 0.9239405393600464),
 ('comfy', 0.923820436000824),
 ('decorated', 0.9085723757743835)]

In [22]:
model2.wv.most_similar("location")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('is', 0.8893594145774841),
 ('perfect', 0.8646179437637329),
 ('neighborhood', 0.8565983176231384),
 ('affordable', 0.8547906279563904),
 ('convenient', 0.8535569906234741),
 ('neighbourhood', 0.8480147123336792),
 ('central', 0.8412215113639832),
 ('great', 0.8313050270080566),
 ('value', 0.8162866234779358),
 ('near', 0.814899206161499)]

In [23]:
model2.wv.most_similar("transport")

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('accessible', 0.9948354363441467),
 ('court', 0.993882417678833),
 ('placa', 0.9926022291183472),
 ('walkable', 0.992347240447998),
 ('beach', 0.9919555187225342),
 ('gloucester', 0.986873209476471),
 ('railway', 0.9866646528244019),
 ('buckingham', 0.9864444732666016),
 ('market', 0.9862738847732544),
 ('rail', 0.9860687255859375)]